In [231]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [232]:
# Create the optimization model
model = gb.Model("Question 3")

In [233]:
nurse_df = pd.read_csv(r"C:\Users\gabri\Downloads\nurse_shift_costs.csv")

In [234]:
nurse_df

,Nurse_ID,Category,Cost_Weekday,Cost_Weekend,Cost_Overtime
0,1,SRN,150,180,248
1,2,RN,360,432,594
2,3,SRN,600,720,990
3,4,RN,120,144,198
4,5,RN,240,288,396
5,6,NIT,320,384,528
6,7,SRN,600,720,990
7,8,NIT,320,384,528
8,9,SRN,150,180,248
9,10,SRN,300,360,495


In [235]:
weekday = nurse_df['Cost_Weekday'].values
weekday

array([150, 360, 600, 120, 240, 320, 600, 320, 150, 300, 300, 160, 120,
       240, 400, 600, 480, 360, 600, 360, 150, 120, 750, 150, 600, 160],
      dtype=int64)

In [236]:
weekend = nurse_df['Cost_Weekend'].values
weekend

array([180, 432, 720, 144, 288, 384, 720, 384, 180, 360, 360, 192, 144,
       288, 480, 720, 576, 432, 720, 432, 180, 144, 900, 180, 720, 192],
      dtype=int64)

In [237]:
overtime = nurse_df['Cost_Overtime'].values
overtime

array([ 248,  594,  990,  198,  396,  528,  990,  528,  248,  495,  495,
        264,  198,  396,  660,  990,  792,  594,  990,  594,  248,  198,
       1238,  248,  990,  264], dtype=int64)

In [238]:
x = model.addVars(26, 10, lb=0, vtype=GRB.BINARY, name="Weekday")
y = model.addVars(26, 4, lb=0, vtype=GRB.BINARY, name="Weekend")
z = model.addVars(26, 14, lb=0, vtype=GRB.BINARY, name="Overtime")

In [239]:
weekday_objective = gb.quicksum(weekday[i]*x[i,j] for i in range(26) for j in range (10))
weekend_objective = gb.quicksum(weekend[i]*y[i,j] for i in range(26) for j in range (4))
overtime_objective = gb.quicksum(overtime[i]*z[i,j] for i in range(26) for j in range (14))
model.setObjective(weekday_objective + weekend_objective + overtime_objective, GRB.MINIMIZE)

In [240]:
for j in range (10):
    model.addConstr(gb.quicksum(x[i,j] for i in range (26)) >= 6, "At least 6 nurses in the week")

In [241]:
for j in range (4):
    model.addConstr(gb.quicksum(y[i,j] for i in range (26)) >= 6, "At least 6 nurses in the weekend")

In [242]:
for i in range(26):
    model.addConstr(gb.quicksum(x[i,j] for j in range(10)) + gb.quicksum(y[i,j] for j in range(4)) >= 3, "At least 36 hours")

for i in range(26):
    model.addConstr(gb.quicksum(x[i,j] for j in range(10)) + gb.quicksum(y[i,j] for j in range(4)) <= 5, "At most 60 hours")

In [243]:
role = nurse_df['Category'].values
role

array(['SRN', 'RN', 'SRN', 'RN', 'RN', 'NIT', 'SRN', 'NIT', 'SRN', 'SRN',
       'SRN', 'NIT', 'RN', 'NIT', 'NIT', 'SRN', 'RN', 'RN', 'RN', 'RN',
       'SRN', 'RN', 'SRN', 'SRN', 'RN', 'NIT'], dtype=object)

In [244]:
SRN_indices = []

for a in role:
    if 'SRN' in a:
        SRN_indices.append(1)
    else:
        SRN_indices.append(0)

SRN_indices

[1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]

In [245]:
for j in range (10):
    model.addConstr(gb.quicksum(SRN_indices[i]*x[i,j] for i in range (26)) >= 1, "At least one SRN")

In [246]:
for j in range (4):
    model.addConstr(gb.quicksum(SRN_indices[i]*y[i,j] for i in range (26)) >= 1, "At least one SRN")

In [247]:
for j in range(9):
    for i in range(26):
        model.addConstr(x[i,j] >= 1 - x[i,j+1], f"No back to backs ({i}, {j})")

In [248]:
for j in range(3):
    for i in range(26):
        model.addConstr(y[i,j] >= 1 - y[i,j+1], f"No back to backs ({i}, {j})")

In [249]:
for j in range(13):
    for i in range(26):
        model.addConstr(z[i,j] >= 1 - z[i,j+1], f"No back to backs ({i}, {j})")

In [250]:
for i in range(26):
    model.addConstr(gb.quicksum(x[i,j] for j in range(10)) + gb.quicksum(y[i,j] for j in range(4)) >= 3 + 3*gb.quicksum(z[i,j] for j in range (14)))

In [251]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 730 rows, 728 columns and 2532 nonzeros
Model fingerprint: 0xbeeb7073
Variable types: 0 continuous, 728 integer (728 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 0 rows and 104 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [253]:
x = model.addVars(26, 10, lb=0, vtype=GRB.BINARY, name="Weekday")
y = model.addVars(26, 4, lb=0, vtype=GRB.BINARY, name="Weekend")
z = model.addVars(26, 14, lb=0, vtype=GRB.BINARY, name="Overtime")

weekday_objective = gb.quicksum(weekday[i] * x[i, j] for i in range(26) for j in range(10))
weekend_objective = gb.quicksum(weekend[i] * y[i, j] for i in range(26) for j in range(4))
overtime_objective = gb.quicksum(overtime[i] * z[i, j] for i in range(26) for j in range(14))
model.setObjective(weekday_objective + weekend_objective + overtime_objective, GRB.MINIMIZE)

for j in range(10):
    model.addConstr(gb.quicksum(x[i, j] for i in range(26)) >= 6, "At least 6 nurses in the week")

for j in range(4):
    model.addConstr(gb.quicksum(y[i, j] for i in range(26)) >= 6, "At least 6 nurses in the weekend")

for i in range(26):
    model.addConstr(gb.quicksum(x[i, j] for j in range(10)) + gb.quicksum(y[i, j] for j in range(4)) >= 3, "At least 36 hours or 3 shifts")

for i in range(26):
    model.addConstr(gb.quicksum(x[i, j] for j in range(10)) + gb.quicksum(y[i, j] for j in range(4)) <= 5, "At most 60 hours or 5 shifts")

for j in range(9):
    for i in range(26):
        model.addConstr(x[i, j] >= 1 - x[i, j + 1], f"No back to backs ({i}, {j})")

for j in range(3):
    for i in range(26):
        model.addConstr(y[i, j] >= 1 - y[i, j + 1], f"No back to backs ({i}, {j})")

for j in range(13):
    for i in range(26):
        model.addConstr(z[i, j] >= 1 - z[i, j + 1], f"No back to backs ({i}, {j})")

for i in range(26):
    model.addConstr(gb.quicksum(x[i, j] for j in range(10)) + gb.quicksum(y[i, j] for j in range(4)) >= 3 + 3 * gb.quicksum(z[i, j] for j in range(14)))

model.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1472 rows, 2184 columns and 5652 nonzeros
Model fingerprint: 0x0d04cfaa


Variable types: 0 continuous, 2184 integer (2184 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 338 rows and 1196 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [256]:
# Objective Function
weekday_objective = gb.quicksum(weekday[i] * x[i, j] for i in range(26) for j in range(10))
weekend_objective = gb.quicksum(weekend[i] * y[i, j] for i in range(26) for j in range(4))
overtime_objective = gb.quicksum(overtime[i] * z[i, j] for i in range(26) for j in range(14))
model.setObjective(weekday_objective + weekend_objective + overtime_objective, GRB.MINIMIZE)

# Constraints
# Minimum and Maximum Working Hours
for i in range(26):
    model.addConstr(gb.quicksum(x[i, j] for j in range(10)) + gb.quicksum(y[i, j] for j in range(4)) + gb.quicksum(z[i, j] for j in range(14)) >= 36, f"At least 36 hours for nurse {i}")
    model.addConstr(gb.quicksum(x[i, j] for j in range(10)) + gb.quicksum(y[i, j] for j in range(4)) + gb.quicksum(z[i, j] for j in range(14)) <= 60, f"At most 60 hours for nurse {i}")

# Nurse Assignment Constraints
for j in range(10):
    model.addConstr(gb.quicksum(x[i, j] for i in range(26)) >= 6, "At least 6 nurses in the week")

for j in range(4):
    model.addConstr(gb.quicksum(y[i, j] for i in range(26)) >= 6, "At least 6 nurses in the weekend")

# Back-to-Back Shifts Constraints
for j in range(9):
    for i in range(26):
        model.addConstr(x[i, j] >= 1 - x[i, j + 1], f"No back to backs weekday ({i}, {j})")

for j in range(3):
    for i in range(26):
        model.addConstr(y[i, j] >= 1 - y[i, j + 1], f"No back to backs weekend ({i}, {j})")

for j in range(13):
    for i in range(26):
        model.addConstr(z[i, j] >= 1 - z[i, j + 1], f"No back to backs overtime ({i}, {j})")

# SRN Constraints
for j in range(10):
    model.addConstr(gb.quicksum(SRN_indices[i] * x[i, j] for i in range(26)) >= 1, "At least one SRN in weekday")

for j in range(4):
    model.addConstr(gb.quicksum(SRN_indices[i] * y[i, j] for i in range(26)) >= 1, "At least one SRN in weekend")

# Overtime Constraints
for i in range(26):
    model.addConstr(gb.quicksum(x[i, j] for j in range(10)) + gb.quicksum(y[i, j] for j in range(4)) + gb.quicksum(z[i, j] for j in range(14)) >= 36 + 3 * gb.quicksum(z[i, j] for j in range(14)), f"Overtime for nurse {i}")

# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2958 rows, 2184 columns and 12900 nonzeros
Model fingerprint: 0x585296fb
Variable types: 0 continuous, 2184 integer (2184 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]


Presolve removed 390 rows and 1092 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
